## Importing Required Packages

In [1]:
import os
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from collections import Counter
from tqdm import tqdm
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\balur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Loading Data Set

In [2]:
data_set = {}

In [3]:
for root, dirs, files in os.walk("C:/Users/balur/OneDrive/Desktop/UTA/Sem-2/ML/Project's/project-2/20_newsgroups"): 
    for d in dirs:
        data_set[d]=[]
        for f in (os.scandir("C:/Users/balur/OneDrive/Desktop/UTA/Sem-2/ML/Project's/project-2/20_newsgroups/"+d)):
            data_set[d].append(f.name)

In [4]:
train_data ={}
test_data ={}

### Classes

In [5]:
data_set.keys()

dict_keys(['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc'])

### Splitting Data Set into train and test 50:50

In [6]:
for class_name in tqdm(data_set.keys()):    
    l = int(len(data_set[class_name])*0.5) 
    random.shuffle(data_set[class_name])
    train_data[class_name] = data_set[class_name][0:l]
    test_data[class_name] = data_set[class_name][l:2*l]

100%|████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 2500.18it/s]


In [7]:
words =[]

In [8]:
tr_words ={}    

In [9]:
words_count = {}

In [10]:
spl_characters = [']',';',':','|','\\','"',"'",'\n','<','>',',','.','?','/','-','*','~','`','!','@','^','$','%','&','*','(',')','+','=','{','}','[']

### Extracting Words 

In [11]:
for i in tqdm(train_data):  
    tr_words[i]=[]
    for j in train_data[i]:
        with open(("C:/Users/balur/OneDrive/Desktop/UTA/Sem-2/ML/Project's/project-2/20_newsgroups//"+i +"//"+j),encoding="utf-8", errors='ignore') as value:
            value = value.read()
            value = value.replace("\n"," ")
            for k in spl_characters:
                value.replace(k," ")    
            tr_words[i].extend(word_tokenize(value))
    words_count[i] = Counter(tr_words[i])
    words.extend(tr_words[i])  
words = set(words)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:23<00:00,  1.15s/it]


In [12]:
print('total words:',(len(words)))

total words: 204092


### dictionary of prob of a word in each class

In [13]:
words = list(words)
dictionary = {}
doc_num = 1
for i in tqdm(words): 
    dictionary[i] = {}
    for j in tr_words: 
        if i in words_count[j]:
            dictionary[i][j] = words_count[j][i]
        else:
            dictionary[i][j] = 0.1
        dictionary[i][j]/=len(tr_words[j])

100%|███████████████████████████████████████████████████████████████████████| 204092/204092 [00:02<00:00, 70312.12it/s]


In [14]:
prob = {}
total= 0

## Calculating probabilities of words in train data set

In [15]:
for i in train_data:
    prob[i] = len(train_data[i])
    total += len(train_data[i])
for j in prob:
    prob[j] = prob[j]/total

In [16]:
correct = 0
total_count = 0

##  Naive Bayes classifier on Test Data Set

In [17]:
for class_name in tqdm(test_data):
    for file_name in test_data[class_name]:
        
        with open(("C:/Users/balur/OneDrive/Desktop/UTA/Sem-2/ML/Project's/project-2/20_newsgroups//"+class_name +"//"+file_name),encoding="utf-8", errors='ignore') as value:
            value = value.read()
            test_words = word_tokenize(value)
            test_words = set(test_words)
            p = {}
            for k in test_data:
                p[k] = prob[k]
                for w in test_words:
                    if w in dictionary:
                        p[k] = p[k]* dictionary[w][k]*1000
                    else:
                        p[k] = p[k]* 0.1/len(tr_words[class_name])*1000
            max_keys = [k for k, v in p.items() if v == max(p.values())]
            correct += (max_keys[0]==class_name)
            total_count+=1

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:41<00:00,  2.08s/it]


In [18]:
print("Accuracy: ", correct*100/total_count) #76.90538107621525 #78.8257651530306

Accuracy:  78.29565913182637
